In [1]:
import os
os.chdir('../src')

In [2]:
from glob import glob

import pandas as pd
import numpy as np

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
from torch.cuda import amp
from torch.utils.data.sampler import SequentialSampler

import timm
from timm.models.resnet import resnet34d

import cv2
import matplotlib.pyplot as plt

from datasets import create_train_loader, create_valid_loader
from configs import cfg

In [3]:
path_df = os.path.join(
        cfg.INPUT.ROOT_DIR, 'train_folds.csv'
    )

df = pd.read_csv(
    path_df
)

train_df = df[df['fold'] != cfg.INPUT.VALID_FOLD]
valid_df = df[df['fold'] == cfg.INPUT.VALID_FOLD]

#Se sono in modalità debug itero velocemente
if cfg.DEBUG:
    train_df = train_df.sample(frac=0.1)
    valid_df = valid_df.sample(frac=0.1)


# Creo i loader
train_loader = create_train_loader(cfg, df=train_df)

In [31]:
class kaggleBCModel(torch.nn.Module):
    def __init__(self, aux_class, cfg) -> None:
        super().__init__()

        self.cfg = cfg
        #TODO: ricordarsi nella prediction di fare un sigmoid dopo l'output
        self.register_buffer('mean', torch.FloatTensor([0.5, 0.5, 0.5]).reshape(1, 3, 1, 1))
        self.register_buffer('std', torch.FloatTensor([0.5, 0.5, 0.5]).reshape(1, 3, 1, 1))
        self.encoder = timm.create_model(
            self.cfg.SOLVER.MODEL_NAME, pretrained=True, in_chans = 3
        )

        self.fe_dim = self.encoder.fc.in_features
        
        self.cancer_layer = torch.nn.Sequential(
            torch.nn.Linear(self.fe_dim, 1),
        )

        self.aux_layer = torch.nn.ModuleList([
            torch.nn.Linear(self.fe_dim, aux_dim) for aux_dim in aux_class
        ])

    def forward(self, x):

        batch_size,C,H,W = x.shape
        x = (x - self.mean) / self.std

        e = self.encoder.forward_features(x)
        x = F.adaptive_avg_pool2d(e, 1)
        x = torch.flatten(x,1,3)
        cancer = self.cancer_layer(x).reshape(-1)

        aux_pred = []
        for layer in self.aux_layer:
            aux_pred.append(layer(x))

        return cancer, aux_pred

model = kaggleBCModel(
    aux_class=train_df[cfg.INPUT.AUX_TARGETS].max()+1,
    cfg=cfg
)

In [32]:
model = model.to(cfg.DEVICE)

In [33]:
model.encoder.fc.in_features

2048

In [34]:
with torch.no_grad():

    for imgs, targets, _, _ in train_loader:
        imgs = imgs.float().to(cfg.DEVICE)
        model(imgs)

KeyboardInterrupt: 

Traceback (most recent call last):
  File "/usr/lib/python3.8/multiprocessing/queues.py", line 245, in _feed
    send_bytes(obj)
  File "/usr/lib/python3.8/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/usr/lib/python3.8/multiprocessing/connection.py", line 411, in _send_bytes
    self._send(header + buf)
  File "/usr/lib/python3.8/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe
